### Installation

In [1]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))

# Install torch geometric
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-geometric
!pip install ogb

PyTorch has version 1.13.1+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Load datasets

In [2]:
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

from ogb.linkproppred import PygLinkPropPredDataset

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
def load_node_dataset(dataset_name):
    # Load the dataset and transform it to sparse tensor
    dataset = PygNodePropPredDataset(name=dataset_name,
                                    transform=T.ToSparseTensor())
    print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))

    # Split
    split_idx = dataset.get_idx_split()

    # Extract the graph
    data = dataset[0]
    data = data.to(device)

    print('Task type: {}'.format(dataset.task_type))

    return dataset, split_idx, data 


In [5]:
# load edge dataset 
d_name = 'ogbl-collab'
dataset = PygLinkPropPredDataset(name = d_name) 

split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0] # pyg graph object containing only training edges



In [6]:
dataset_arxiv, split_arxiv, data_arxiv = load_node_dataset(dataset_name='ogbn-arxiv')

The ogbn-arxiv dataset has 1 graph
Task type: multiclass classification


### Baseline GNNs

In [7]:
import torch
import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
from torch_geometric.nn import Sequential, GCNConv

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import BatchNorm1d, Dropout, Linear, Parameter, ReLU
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax, degree

from torch_geometric.nn import GCNConv,GATConv,GATv2Conv, GINConv
from matplotlib import pyplot as plt
import pandas as pd

import copy
import pickle

In [8]:
def train(model, data, train_idx, optimizer, loss_fn):
      model.train()
      loss = 0
      optimizer.zero_grad()
      out = model(data.x, data.adj_t)
      loss = loss_fn(out[train_idx], data.y[train_idx][:, 0])
      loss.backward()
      optimizer.step()

      return loss.item()

In [9]:
@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    model.eval()
    out = model(data.x, data.adj_t)

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      # Save locally as csv
      df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

In [10]:
def evaluate_model(model, args):
  model.reset_parameters()

  evaluator = Evaluator(name='ogbn-arxiv')
  optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
  loss_fn = F.nll_loss

  best_model = None
  best_valid_acc = 0

  train_accs, valid_accs, test_accs = [], [], []
  for epoch in range(1, 1 + args['epochs']):
    loss = train(model, data_arxiv, split_arxiv['train'], optimizer, loss_fn)
    result = test(model, data_arxiv, split_arxiv, evaluator)
    train_acc, valid_acc, test_acc = result
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)
    test_accs.append(test_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    if (epoch < 5) or (epoch % 10 == 0):
      print(f'Epoch: {epoch:02d}, '
      f'Loss: {loss:.4f}, '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')
  # plot losses in matplotlib
  plt.plot(train_accs, label='train')
  plt.plot(valid_accs, label='valid')
  plt.plot(test_accs, label='test')
  plt.legend()
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.title(f"{model.name} obgn-arxiv node classification")
  plt.savefig(f"{model.name}-accuracy.png")
  plt.show()



In [11]:
class GCN(torch.nn.Module):
  # 3-layer GCN
  def __init__(self, input_dim, hidden_dim, output_dim, dropout):
    super(GCN, self).__init__()
    self.name = "GCN"
    self.model = Sequential('x, edge_index', [
        (GCNConv(input_dim, hidden_dim), 'x, edge_index -> x'),
        BatchNorm1d(hidden_dim),
        ReLU(),
        Dropout(p=dropout),
        (GCNConv(hidden_dim, hidden_dim), 'x, edge_index -> x'),
        BatchNorm1d(hidden_dim),
        ReLU(),
        Dropout(p=dropout),
        (GCNConv(hidden_dim, hidden_dim), 'x, edge_index -> x'),
        BatchNorm1d(hidden_dim),
        ReLU(),
        Dropout(p=dropout),
        (GCNConv(hidden_dim, output_dim), 'x, edge_index -> x'),
        nn.LogSoftmax(1),
    ])
  
  def reset_parameters(self):
    for layer in self.model.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
  
  def forward(self, x, adj_t):
    return self.model(x, adj_t)

In [12]:
gcn_args = {
  'device': device,
  'hidden_dim': 256,
  'dropout': 0.5,
  'lr': 0.01,
  'epochs': 100,
}
gcn = GCN(dataset_arxiv.num_features, gcn_args['hidden_dim'], dataset_arxiv.num_classes, dropout=gcn_args['dropout']).to(device)

# evaluate_model(gcn, gcn_args)

In [13]:
torch.cuda.empty_cache()

In [14]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, concat, dropout):
        super(GAT, self).__init__()
        self.model = Sequential('x, edge_index', [
            (GATConv(input_dim, hidden_dim, heads=num_heads, concat=concat), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GATConv(hidden_dim, hidden_dim, heads=num_heads, concat=concat), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GATConv(hidden_dim, hidden_dim, heads=num_heads, concat=concat), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GATConv(hidden_dim, output_dim, heads=num_heads, concat=concat), 'x, edge_index -> x'),
            nn.LogSoftmax(1),
        ])
    
    def reset_parameters(self):
        for layer in self.model.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
    
    def forward(self, x, adj_t):
        return self.model(x, adj_t)

In [15]:
gat_args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'num_heads': 8,
    'concat': True,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
gat = GAT(dataset_arxiv.num_features, gat_args['hidden_dim'], dataset_arxiv.num_classes, gat_args['num_heads'], gat_args['concat'], dropout=gat_args['dropout']).to(device)
evaluate_model(gat, gat_args)

OutOfMemoryError: ignored

In [ ]:
class GIN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout):
        super(GIN, self).__init__()
        self.name = "GIN"
        self.model = Sequential('x, edge_index', [
            (GINConv(Sequential('x, edge_index', [
                (GCNConv(input_dim, hidden_dim), 'x, edge_index -> x'),
                BatchNorm1d(hidden_dim),
                ReLU(),
                Dropout(p=dropout),
            ])), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GINConv(Sequential('x, edge_index', [
                (GCNConv(hidden_dim, hidden_dim), 'x, edge_index -> x'),
                BatchNorm1d(hidden_dim),
                ReLU(),
                Dropout(p=dropout),
            ])), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GINConv(Sequential('x, edge_index', [
                (GCNConv(hidden_dim, hidden_dim), 'x, edge_index -> x'),
                BatchNorm1d(hidden_dim),
                ReLU(),
                Dropout(p=dropout),
            ])), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GCNConv(hidden_dim, output_dim), 'x, edge_index -> x'),
            nn.LogSoftmax(1),
        ])
    
    def reset_parameters(self):
        for layer in self.model.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
    
    def forward(self, x, adj_t):
        return self.model(x, adj_t)

In [ ]:
gin_args = {
    'device': device,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
gin = GIN(dataset_arxiv.num_features, gin_args['hidden_dim'], dataset_arxiv.num_classes, dropout=gin_args['dropout']).to(device)

evaluate_model(gin, gin_args)

### graveyard


In [ ]:
# from torch_geometric.datasets import OGB_MAG

# dataset_mag = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
# data_mag = dataset[0]

# from torch_geometric.nn import SAGEConv, to_hetero

# class GNN(torch.nn.Module):
#     def __init__(self, hidden_channels, out_channels):
#         super().__init__()
#         self.conv1 = SAGEConv((-1, -1), hidden_channels)
#         self.conv2 = SAGEConv((-1, -1), out_channels)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index).relu()
#         x = self.conv2(x, edge_index)
#         return x


# model = GNN(hidden_channels=64, out_channels=dataset.num_classes)
# model = to_hetero(model, data_mag.metadata(), aggr='sum')

# node_types = ['paper', 'author']
# edge_types = [
#     ('paper', 'cites', 'paper'),
#     ('paper', 'written_by', 'author'),
#     ('author', 'writes', 'paper'),
# ]
# metadata = (node_types, edge_types)

# from torch_geometric.nn import GCNConv

# class GCN(torch.nn.Module):
#   def __init__(self, input_dim, hidden_dim, output_dim):
#     self.conv1 = GCNConv(in_channels=input_dim, out_channels=hidden_dim)
#     self.conv2 = GCNConv(in_channels=hidden_dim, out_channels=hidden_dim)

# # NOTE: idk rly if we're allowed to use colab code because it seems to me that these will be public (?) after the class and idk if they want official solutions/implementations floating around the internet? i'll make a post on ed

### Graph CL Augmentations
